In [10]:
import numpy as np
from itertools import combinations

np.random.seed(42)

N = 100
p = 8
X = np.random.normal(0, 1, (N, p))
beta_true = np.array([2.0, -1.5, 3.0, 0, 0, 0, 0, 0])
intercept_true = 1.0
y = intercept_true + X @ beta_true + np.random.normal(0, 2.0, N)

X_full = np.column_stack((np.ones(N), X))  # (100, 9)

In [6]:
def forward_stepwise(X, y, max_k=None):
    """
    Forward stepwise selection.
    Tecnología inversa: empezamos con intercepto solo, agregamos una variable a la vez
    que más reduzca RSS.
    """
    n, p = X.shape
    if max_k is None:
        max_k = p - 1
    
    current_cols = [0]  # intercepto siempre
    remaining_cols = list(range(1, p))
    results = []
    
    for k in range(1, max_k + 1):
        best_col = None
        best_rss = np.inf
        
        for col in remaining_cols:
            cols_try = current_cols + [col]
            X_try = X[:, cols_try]
            betas = np.linalg.lstsq(X_try, y, rcond=None)[0]
            y_pred = X_try @ betas
            rss = np.sum((y - y_pred)**2)
            if rss < best_rss:
                best_rss = rss
                best_col = col
        
        current_cols.append(best_col)
        remaining_cols.remove(best_col)
        
        # Criterios
        k_total = len(current_cols)
        aic = n * np.log(best_rss / n) + 2 * k_total
        bic = n * np.log(best_rss / n) + np.log(n) * k_total
        
        results.append({
            'k': k,
            'rss': best_rss,
            'aic': aic,
            'bic': bic,
            'cols': current_cols[1:]  # sin intercepto
        })
    
    return results

In [9]:
# Forward stepwise
stepwise_results = forward_stepwise(X_full, y, max_k=5)

print("Forward Stepwise:")
print("k | RSS | AIC | BIC | Predictores")
print("-" * 50)
for r in stepwise_results:
    print(f"{r['k']} | {r['rss']:.2f} | {r['aic']:.2f} | {r['bic']:.2f} | {r['cols']}")

# Comparación con best subset (copia la tabla del notebook anterior o ejecútala de nuevo)
print("\nBest Subset (del ejercicio anterior):")

# Pega aquí tu tabla anterior para comparar

Forward Stepwise:
k | RSS | AIC | BIC | Predictores
--------------------------------------------------
1 | 760.18 | 206.84 | 212.05 | [3]
2 | 470.63 | 160.89 | 168.71 | [3, 1]
3 | 339.94 | 130.36 | 140.78 | [3, 1, 2]
4 | 333.65 | 130.49 | 143.52 | [3, 1, 2, 7]
5 | 331.77 | 131.93 | 147.56 | [3, 1, 2, 7, 8]

Best Subset (del ejercicio anterior):


## Comparación: Best Subset vs Forward Stepwise

*Best Subset Selection:*

| k | RSS    | Cp    | AIC   | BIC   | Mejores predictores |
|---|--------|-------|-------|-------|---------------------|
| 1 | 760.18 | 42.14 | 206.84| 212.05| (3,)                |
| 2 | 470.63 | 26.19 | 160.89| 168.71| (1, 3)              |
| 3 | 339.94 | 20.08 | 130.36| 140.78| (1, 2, 3)           |
| 4 | 333.65 | 21.69 | 130.49| 143.52| (1, 2, 3, 7)        |
| 5 | 331.77 | 23.57 | 131.93| 147.56| (1, 2, 3, 7, 8)     |

*Forward Stepwise:*

| k | RSS    | AIC   | BIC   | Predictores |
|---|--------|-------|-------|-------------|
| 1 | 760.18 | 206.84| 212.05| [3]         |
| 2 | 470.63 | 160.89| 168.71| [3, 1]      |
| 3 | 339.94 | 130.36| 140.78| [3, 1, 2]   |
| 4 | 333.65 | 130.49| 143.52| [3, 1, 2, 7]|
| 5 | 331.77 | 131.93| 147.56| [3, 1, 2, 7, 8] |

### Observaciones y conclusiones

- *Best Subset* encontró el modelo óptimo en k=3 con los predictores verdaderos (1,2,3).  
- *Forward Stepwise* eligió exactamente los mismos en k=3 → en este caso coincidieron.  
- En datos con ruido o multicolinealidad, stepwise puede desviarse del óptimo (greedy).  
- AIC/BIC/Cp penalizan complejidad → evitan k=4/5 (agregan ruido).  

Este ejercicio muestra que stepwise es una buena aproximación rápida, pero best subset es el gold standard cuando p no es muy grande